# Classify Dog and Cat breeds using Transfer Learning

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # The %tensorflow_version magic only works in colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
tf.__version__

'1.14.0'

## Setup Input Pipeline

Download the flowers dataset.

In [4]:
#_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

#zip_file = tf.keras.utils.get_file(origin=_URL, 
 #                                  fname="flower_photos.tgz", 
  #                                 extract=True)
current_path = os.getcwd()
data_directory = 'images'
base_dir = os.path.join(current_path, data_directory, '')
print(base_dir)

C:\Users\vaibh\Documents\images\


Use `ImageDataGenerator` to rescale the images.

Create the train generator and specify where the train dataset directory, image size, batch size.

Create the validation generator with similar approach as the train generator with the flow_from_directory() method.

In [5]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 5913 images belonging to 35 classes.
Found 1477 images belonging to 35 classes.


In [6]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 35))

Save the labels in a file which will be downloaded later.

In [7]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'Abyssinian': 0, 'Bengal': 1, 'Birman': 2, 'Bombay': 3, 'British': 4, 'Egyptian': 5, 'Maine': 6, 'Persian': 7, 'Ragdoll': 8, 'Russian': 9, 'Siamese': 10, 'Sphynx': 11, 'american': 12, 'basset': 13, 'beagle': 14, 'boxer': 15, 'chihuahua': 16, 'english': 17, 'german': 18, 'great': 19, 'havanese': 20, 'japanese': 21, 'keeshond': 22, 'leonberger': 23, 'miniature': 24, 'newfoundland': 25, 'pomeranian': 26, 'pug': 27, 'saint': 28, 'samoyed': 29, 'scottish': 30, 'shiba': 31, 'staffordshire': 32, 'wheaten': 33, 'yorkshire': 34}


In [8]:
!cat labels.txt

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Create the base model from the pre-trained convnets

Create the base model from the **MobileNet V2** model developed at Google, and pre-trained on the ImageNet dataset, a large dataset of 1.4M images and 1000 classes of web images.

First, pick which intermediate layer of MobileNet V2 will be used for feature extraction. A common practice is to use the output of the very last layer before the flatten operation, the so-called "bottleneck layer". The reasoning here is that the following fully-connected layers will be too specialized to the task the network was trained on, and thus the features learned by these layers won't be very useful for a new task. The bottleneck features, however, retain much generality.

Let's instantiate an MobileNet V2 model pre-loaded with weights trained on ImageNet. By specifying the `include_top=False` argument, we load a network that doesn't include the classification layers at the top, which is ideal for feature extraction.

In [9]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## Feature extraction
You will freeze the convolutional base created from the previous step and use that as a feature extractor, add a classifier on top of it and train the top-level classifier.

In [10]:
base_model.trainable = False

### Add a classification head

In [16]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(35, activation='softmax')
])

### Compile the model

You must compile the model before training it.  Since there are two classes, use a binary cross-entropy loss.

In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 35)                1155      
Total params: 2,627,811
Trainable params: 369,827
Non-trainable params: 2,257,984
_________________________________________________________________


In [19]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 4


### Train the model

<!-- TODO(markdaoust): delete steps_per_epoch in TensorFlow r1.14/r2.0 -->

In [20]:
epochs = 10

history = model.fit_generator(train_generator, 
                    epochs=epochs, 
                    validation_data=val_generator)

Epoch 1/10
93/93 [==============================] - 290s 3s/step - loss: 2.0109 - acc: 0.4414 - val_loss: 1.1278 - val_acc: 0.6506
Epoch 2/10
93/93 [==============================] - 72s 774ms/step - loss: 0.8374 - acc: 0.7624 - val_loss: 0.9251 - val_acc: 0.7319
Epoch 3/10
93/93 [==============================] - 37s 396ms/step - loss: 0.5932 - acc: 0.8212 - val_loss: 1.2215 - val_acc: 0.7007
Epoch 4/10
93/93 [==============================] - 38s 404ms/step - loss: 0.4979 - acc: 0.8441 - val_loss: 0.9624 - val_acc: 0.7461
Epoch 5/10
93/93 [==============================] - 39s 419ms/step - loss: 0.3931 - acc: 0.8754 - val_loss: 0.8814 - val_acc: 0.7637
Epoch 6/10
93/93 [==============================] - 38s 405ms/step - loss: 0.3269 - acc: 0.8967 - val_loss: 1.1670 - val_acc: 0.7278
Epoch 7/10
93/93 [==============================] - 37s 394ms/step - loss: 0.3180 - acc: 0.8992 - val_loss: 1.0010 - val_acc: 0.7624
Epoch 8/10
93/93 [==============================] - 37s 400ms/step - lo

### Learning curves

Let's take a look at the learning curves of the training and validation accuracy/loss when using the MobileNet V2 base model as a fixed feature extractor. 

In [24]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

KeyError: 'val_accuracy'

## Fine tuning
In our feature extraction experiment, you were only training a few layers on top of an MobileNet V2 base model. The weights of the pre-trained network were **not** updated during training.

One way to increase performance even further is to train (or "fine-tune") the weights of the top layers of the pre-trained model alongside the training of the classifier you added. The training process will force the weights to be tuned from generic features maps to features associated specifically to our dataset.

### Un-freeze the top layers of the model


All you need to do is unfreeze the `base_model` and set the bottom layers be un-trainable. Then, recompile the model (necessary for these changes to take effect), and resume training.

In [22]:
base_model.trainable = True

In [23]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  155


### Compile the model

Compile the model using a much lower training rate.

In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 35)                1155      
Total params: 2,627,811
Trainable params: 2,232,419
Non-trainable params: 395,392
_________________________________________________________________


In [28]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 60


### Continue Train the model

In [29]:
history_fine = model.fit_generator(train_generator, 
                         epochs=5,
                         validation_data=val_generator)

Epoch 1/5
93/93 [==============================] - 48s 521ms/step - loss: 0.1415 - acc: 0.9564 - val_loss: 1.0286 - val_acc: 0.7739
Epoch 2/5
93/93 [==============================] - 39s 415ms/step - loss: 0.0996 - acc: 0.9723 - val_loss: 1.0329 - val_acc: 0.7752
Epoch 3/5
93/93 [==============================] - 38s 409ms/step - loss: 0.0860 - acc: 0.9760 - val_loss: 0.9979 - val_acc: 0.7840
Epoch 4/5
93/93 [==============================] - 38s 410ms/step - loss: 0.0743 - acc: 0.9816 - val_loss: 0.9977 - val_acc: 0.7793
Epoch 5/5
93/93 [==============================] - 37s 402ms/step - loss: 0.0726 - acc: 0.9838 - val_loss: 1.0447 - val_acc: 0.7779


## Convert to TFLite

Saved the model using `tf.saved_model.save` and then convert the saved model to a tf lite compatible format.

In [31]:
saved_model_dir = 'save/fine_tuning'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

ValueError: None values not supported.

Download the converted model and labels